In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();

 Activating environment at `C:\Users\Zero\Documents\GitHub\16-745-hw1\Project.toml`


In [2]:
using Random, LinearAlgebra

## 一、定义问题结构和维度信息
$$ \begin{align}
    &\text{minimize}_{x} && \frac{1}{2} x^T P x + q^T x \\
    &\text{subject to} && A x = b \\
    &&& C x \leq d \\
\end{align} $$

In [3]:
struct QPData
    P::Matrix{Float64}
    q::Vector{Float64}
    A::Matrix{Float64}
    b::Vector{Float64}
    C::Matrix{Float64}
    d::Vector{Float64}
end

In [4]:
function QPData(n::Int, m::Int, p::Int)
    QPData(zeros(n,n), zeros(n), zeros(m,n), zeros(m), zeros(p,n), zeros(p))
end

QPData

In [5]:
num_eq(qp::QPData) = length(qp.b)
num_ineq(qp::QPData) = length(qp.d)

num_ineq (generic function with 1 method)

In [6]:
Base.size(qp::QPData) = (length(qp.q), num_eq(qp), num_ineq(qp))

In [7]:
objective(qp::QPData, x) = 0.5 * x'qp.P*x + qp.q'x
ceq(qp::QPData, x) = qp.A * x - qp.b
cin(qp::QPData, x) = qp.C * x - qp.d

cin (generic function with 1 method)

In [8]:
function primal_residual(qp::QPData, x, λ, μ)
    qp.P*x + qp.q + qp.A'λ + qp.C'μ
end

primal_residual (generic function with 1 method)

In [9]:
function dual_residual(qp::QPData, x, λ, μ)
    g = ceq(qp, x)
    h = cin(qp, x)
    return [g; max.(0, h)]
end

dual_residual (generic function with 1 method)

In [10]:
function complimentarity(qp::QPData, x, λ, μ)
    return [min.(0, μ); μ .* cin(qp, x)]
end

complimentarity (generic function with 1 method)

## 二、拉格朗日到增广拉格朗日

原问题可以转化为：
$$
\min_x \max_{\lambda, u \ge 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) + u^T (Cx - d) \qquad \text{问题1}
$$

仔细看内部问题
$$
\max_{\lambda, u \ge 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) + u^T (Cx - d) \qquad \text{问题2}
$$

- 如果 $Cx-d < 0$，那么$u$只能为0，所以问题2变成了
$$\max_{\lambda, u = 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) $$
- 如果 $Cx-d \ge 0$，问题2变成了
$$\max_{\lambda, u > 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) + u^T (Cx - d)$$

如果$(Ax-b) \neq 0$或者$(Cx - d)> 0$，那么内部问题就是无穷的，这就迫使约束满足了，所以问题1和原问题是等价的。那么如何求解问题2，由于有一个$\max$操作，非光滑的，不好直接求解。所以用近端点法，就是给一个先验值，然后使得不能离太远，这样就会形成一个关于$\lambda, u$的二阶最大化问题

$$
\max_{\lambda, u \ge 0} \quad \frac{1}{2} x^T P x + q^T x + \lambda^T (Ax-b) - \frac{1}{2 \rho} \| \lambda - \bar{\lambda} \|^2 + u^T (Cx - d) - \frac{1}{2 \rho} \| u - \bar{u} \|^2\qquad \text{问题3}
$$

问题3解的条件是：
$$
\lambda = \bar{\lambda} + \rho (Ax - b) \quad \text{条件1} \\ 
u = \max(\bar{u} + \rho (Cx -d ), 0 )  \quad \text{条件2}
$$

条件2 可以写为
$$
u = \left \{
\begin{aligned}
\bar{u} + \rho (Cx -d ), \ & \text{if} \quad Cx -d \ge 0 \\
0, \ & \text{if} \quad  u < 0
\end{aligned}
\right.
$$

把条件1和2带回问题3，使得问题1变为：
$$
\min_x \quad \frac{1}{2} x^T P x + q^T x + \bar{\lambda}^T (Ax-b)+\frac{\rho}{2} \| Ax - b \|^2 
$$

